In [1]:
import jinja2

In [2]:
import json
from collections import defaultdict
import os
import os.path

Tree = lambda: defaultdict(lambda: Tree())
data = json.load(open('gasapi3.json'))
JS_TYPES_PS_IMPORTS = {
    'void': (("Prelude", ), "Unit"),
    'String': None,
    'Integer': None,
    'Object': (("Foreign", ), "Foreign"),
    'Boolean': None,
    'Byte': (("Foreign", ), "Foreign"),
    'Number': None,
    'Date': (("Data", "JSDate"), "JSDate"),
    'BigNumber': (("Foreign", ), "Foreign"),
    'TargetAudience': (("Foreign", ), "Foreign"),
    'TimeInterval': (("Foreign", ), "Foreign"),
    'Char': None}
TOP_LEVEL = {
    # Apps
    "CalendarApp",
    "ContactsApp",
    "DataStudioApp",
    "DocumentApp",
    "DriveApp",
    "FormApp",
    "GmailApp",
    "GroupsApp",
    "LanguageApp",
    "MailApp",
    "ScriptApp",
    "SitesApp",
    "SlidesApp",
    "SpreadsheetApp",
    "UrlFetchApp",
    "UrlFetchApp",
    
    # Services
    "CacheService",
    "CardService",
    "ConferenceDataService",
    "ContentService",
    "HtmlService",
    "HtmlService",
    "LinearOptimizationService",
    "LockService",
    "LockService",
    "PropertiesService",
    "XmlService",
    
    # Others
    "Charts",
    "Utilities",
    "Maps"

}

In [3]:
def render_js_method(m, c, base_type):
    c = str(c) if c>1 else ""
    base_object = js_type_to_param_name(base_type)
    name = m['name']
    wparams = ", ".join([p['name'] for p in m['parameters']] + [base_object])
    fparams = ", ".join([p['name'] for p in m['parameters']])
    pssign = " -> ".join([js_type_to_ps_type(p['type']) for p in m['parameters']] + [base_type, f"Effect {js_type_to_ps_type(m['result'])}"])
    return f"""
exports.{name}{c}Impl = function({wparams}) {{
    return function () {{
        return {base_object}.{name}({fparams});
    }}
}}  // {pssign}
"""

In [4]:
def js_type_to_param_name(p):
    return "_" + p.lower()

In [5]:
def js_type_to_ps_type(t):
    if t.endswith('[]'):
        return f"(Array {js_type_to_ps_type(t[:-2])})"
    return {
        "void": "Unit",
        "Integer": "Int",
        "Object": "Foreign",
        "Date": "JSDate",
        "TargetAudience": "Foreign",
        "TimeInterval": "Foreign",
        "Byte": "Foreign",
        "BigNumber": "Foreign"
    }.get(t, to_ps_type(t))

In [6]:
def string_to_ps_doc(s):
    return '\n'.join('-- | ' + d for d in s.split('\n'))

In [7]:
def render_ps_funcs(m, c, base_type):
    c = str(c) if c>1 else ""
    base_object = js_type_to_param_name(base_type)
    name = m['name']
    pstypes = list(map(js_type_to_ps_type, [p['type'] for p in m['parameters']] + [base_type, m['result']]))
    pssign = " -> ".join(pstypes[:-1] + [f"Effect {pstypes[-1]}"])
    doc = string_to_ps_doc(m['description'])
    return f"""

foreign import {name}{c}Impl :: EffectFn{len(pstypes) - 1} {" ".join(pstypes)}

{doc}
{name}{c} :: {pssign}
{name}{c} = runEffectFn{len(pstypes) - 1} {name}{c}Impl
"""
# print(render_ps_funcs(data[3]['methods'][4], 1, 'UrlField'))

In [8]:
def get_parts_from_url(url):
    if '#' in url:
        url = url[:url.index('#')]
    if url.endswith('.html'):
        url = url[:-len('.html')]
    parts = url.split('/')
    parts = parts[parts.index('reference')+1:]
    return tuple(parts)

In [9]:
def to_camel_case(s):
    return ''.join(word.title() for word in s.split('-'))

In [10]:
def constant_to_camel_case(s):
    s=''.join(word.title() for word in s.split('_'))
    return s[0].lower() + s[1:]

In [11]:
def constant_to_full_camel_case(s):
    return ''.join(word.title() for word in s.split('_'))

In [12]:
def to_module(ps):
    return '.'.join(to_camel_case(p) for p in ps)

In [13]:
RESERVED = {"Type"}

def to_ps_type(s):
    s = s[0].upper() + s[1:]
    if s in RESERVED:
        s = s+"_"
    return s

In [14]:
def manual_resolve_import(t, base, imports, pb):
    if t == "Position":
        return tuple(pb) + ("Charts", "Position", "Type")
    if t == "Document":
        return tuple(pb) + ("Document", "Document", "Type")


In [15]:
def get_needed_imports_for_type(t, base, imports, pb):
    if t.endswith('[]'):
        return get_needed_imports_for_type(t[:-2], base, imports, pb)
    if t.endswith('...'):
        return get_needed_imports_for_type(t[:-3], base, imports, pb)
    if t in JS_TYPES_PS_IMPORTS:
        return JS_TYPES_PS_IMPORTS[t]
    if t.upper() == base[-1].upper():
        return  # No need to import this very module
    idict = {k.upper(): k for k in imports.keys()}
    if t.upper() in idict:
        lp = idict[t.upper()]
        parent_candidates = set(imports[idict[t.upper()]].keys())
        if not parent_candidates:
            print(f"No candidate imports for {t} in {base} with imports={list(sorted(imports.keys()))}")
            return (tuple(), t)
        elif len(parent_candidates) == 1:  # Only choice wins
            return (tuple(pb) + tuple(parent_candidates) + (lp, "Type"), t)
        if base[-2].upper() in [s.upper() for s in parent_candidates]:  # Same parent is best guess
            return (tuple(pb) + (base[-2], lp, "Type"), t)
        elif "Base" in parent_candidates:  # Is maybe in Base?
            return (tuple(pb) + ("Base", lp, "Type"), t)
        elif "Utilities" in parent_candidates:  # Maybe in Utilities?
            return (tuple(pb) + ("Utilities", lp, "Type"), t)
        elif (imp := manual_resolve_import(t, base, imports, pb)) is not None:
            return (imp, t)
        else:
            print(f"I don't know where {t} is for {base} where parents are {parent_candidates}")
            return (tuple(), t)
    else:
        print(f"Type definition not found for {t} in {base}")
        return (tuple(), t)

In [16]:
def render_class(baseparts, e, imports):
    parts = get_parts_from_url(e['url'])
    filespath = os.path.join(*baseparts, *map(to_camel_case, parts))
    ps_methods_imports = f"""module {'.'.join([*baseparts, to_module(parts + ('Methods', ))])} where
import Effect.Uncurried
import Effect (Effect)

import {'.'.join([*baseparts, to_module(parts + ('Type', ))])} ({to_ps_type(e['name'])})

"""
    ps_methods = ""
    ps_type = f"module {'.'.join([*baseparts, to_module(parts + ('Type', ))])} where\n\n"
    js_methods = '"use strict";\n'

    cs = defaultdict(lambda: 0)
    needed_imports = set()
    for m in e['methods']:
        if any('...' in p['type'] for p in m['parameters']):
            continue
        cs[m['name']] += 1
        js_methods += render_js_method(m, cs[m['name']], e['name'])
        ps_methods += render_ps_funcs(m, cs[m['name']], e['name'])
        for t in [p['type'] for p in m['parameters']] + [m['result']]:
            if (imp := get_needed_imports_for_type(t, tuple(map(to_camel_case, parts)), imports, baseparts)) is not None:
                needed_imports.add(imp)

    for (moduleparts, name) in sorted(needed_imports):
        if moduleparts:
            ps_methods_imports += f"import {'.'.join(moduleparts)} ({to_ps_type(name)}, {name.lower()}PS2JS, {name.lower()}JS2PS)\n"
        else:
            ps_methods_imports += f"-- TODO: Add missing import for type ({name})\n"

    ps_type += f"""
foreign import data {to_ps_type(e['name'])} :: Type
"""
    result = {os.path.join(filespath, 'Methods.js'): js_methods,
              os.path.join(filespath, 'Methods.purs'): ps_methods_imports + ps_methods,
             }
    if e['name'] in TOP_LEVEL:
        js_type = f'''"use strict";
exports.{e['name'].lower()} = {e['name']};
'''
        ps_type += f'''foreign import {e['name'].lower()} :: {to_ps_type(e['name'])}
'''
        result[os.path.join(filespath, 'Type.js')] = js_type

    result[os.path.join(filespath, 'Type.purs')] = ps_type
    
    return result

In [17]:
def render_ps_enum(e, enums_parents):
    fn_name = f"{constant_to_camel_case(e['name'])}Members"
    type_name = e['name']
    elems = ', '.join(constant_to_full_camel_case(p['name']) for p in e['properties'])
    return jinja2.Template(
"""foreign import {{constant_to_camel_case(e['name'])}}JS2PSImpl :: Fn3 (Array {{type_name}}) {{e['name']}}Impl {{type_name}}
{{constant_to_camel_case(e['name'])}}JS2PS :: {{e['name']}}Impl -> {{type_name}}
{{constant_to_camel_case(e['name'])}}JS2PS = runFn3 {{constant_to_camel_case(e['name'])}}JS2PSImpl {{fn_name}}
where
  {{fn_name}} :: Array {{type_name}}
  {{fn_name}} = [{{elems}}]

{% for p in e['properties'] -%}
foreign import data {{constant_to_camel_case(p['name'])}} :: {{e['name']}}Impl
{% endfor %}

{{constant_to_camel_case(e['name'])}}PS2JS :: {{type_name}} -> {{e['name']}}Impl
{%- for p in e['properties'] %}
{{constant_to_camel_case(e['name'])}}PS2JS {{constant_to_full_camel_case(p['name'])}} = {{constant_to_camel_case(p['name'])}}
{%- endfor %}
""").render(e=e, type_name=type_name, fn_name=fn_name, elems=elems, constant_to_camel_case=constant_to_camel_case, constant_to_full_camel_case=constant_to_full_camel_case)

In [18]:
def render_js_enum(p, e, enums_parents):
    parts = get_parts_from_url(e['url'])
    parent = enums_parents.get(parts, {})
    if parent and parent['parent_name'] in TOP_LEVEL:
        return f"""
exports.{constant_to_camel_case(p['name'])} = {parent['parent_name']}.{e['name']}.{p['name']}
"""
    else:
        return ""

In [19]:
def render_ps_enum_type(e):
    constructors = '\n  | '.join(constant_to_full_camel_case(p['name']) for p in e['properties'])
    return f"""data {e['name']} =
    {constructors}
"""

In [20]:
def enum_js_to_ps(e, enums_parents):
    parts = get_parts_from_url(e['url'])
    parent = enums_parents.get(parts, {})
    if parent and parent['parent_name'] in TOP_LEVEL:
        return jinja2.Template("""
export.{{constant_to_camel_case(e['name'])}}JS2PSImpl = function (a, v) {
    switch (v) {
    {%- for p in e['properties'] %}
      case {{parent}}.{{e['name']}}.{{p['name']}}:
        return a[{{loop.index0}}];
    {% endfor -%}
    }
}""").render(e=e, parent=parent['parent_name'], constant_to_camel_case=constant_to_camel_case)
    else:
        print(f"No way! {e['name']}: {e['url']}")
        return jinja2.Template("""
export.{{constant_to_camel_case(e['name'])}}JS2PSImpl = function (a, p, v) {
    switch (v) {
    {%- for p in e['properties'] %}
      case p.{{e['name']}}.{{p['name']}}:
        return a[{{loop.index0}}];
    {% endfor -%}
    }
}""").render(e=e, constant_to_camel_case=constant_to_camel_case)

In [21]:
def render_enum(baseparts, e, enums_parents):
    parts = get_parts_from_url(e['url'])
    filespath = os.path.join(*baseparts, *map(to_camel_case, parts))
    type_module_name = '.'.join([*baseparts, to_module(parts + ('Type',))])
    ps_type = f"""module {type_module_name} where

foreign import data {to_ps_type(e['name'])}Impl :: Type

{render_ps_enum_type(e)}

{render_ps_enum(e, enums_parents)}

"""
    js = f'''
"use strict";

{enum_js_to_ps(e, enums_parents)}


'''

    return {os.path.join(filespath, 'Type.js'): js,
            os.path.join(filespath, 'Type.purs'): ps_type}

In [22]:
def render_entry(baseparts, e, enums_parents, imports):
    if e['type'] in ("class", "interface"):
        return render_class(baseparts, e, imports)
    elif e['type'] == "enum":
        return render_enum(baseparts, e, enums_parents)
    else:
        return None

In [23]:
def get_enums_parents(es):
    for e in es:
        if e['type'] not in ('class', 'interface'):
            continue
        for p in e['properties']:
            yield (get_parts_from_url(p['url']), {'parent_url': get_parts_from_url(e['url']), 'parent_name': e['name'], 'parent_type': e['type'], **p})

In [24]:
def get_imports_backtree(es):
    tr = t = Tree()
    for e in es:
        parts = map(to_camel_case, get_parts_from_url(e['url']))
        for p in reversed(list(parts)):
            t = t[p]
        t = tr
    return tr

In [25]:
def render_entries(baseparts, es):
    enums_parents = dict(get_enums_parents(data))
    imports = get_imports_backtree(data)
    files = dict()
    for e in es:
        
        if (r := render_entry(baseparts, e, enums_parents, imports)) is not None:
            files.update(r)
    return files

In [26]:
files = render_entries(('Google', 'AppsScript'), data)
for filename, content in files.items():
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w") as f:
        f.write(content)

No way! PickerValuesLayout: https://developers.google.com/apps-script/reference/charts/picker-values-layout
No way! MatchType: https://developers.google.com/apps-script/reference/charts/match-type
No way! Orientation: https://developers.google.com/apps-script/reference/charts/orientation
No way! SwitchControlType: https://developers.google.com/apps-script/reference/card-service/switch-control-type
No way! DisplayStyle: https://developers.google.com/apps-script/reference/card-service/display-style
No way! Button: https://developers.google.com/apps-script/reference/base/button
No way! ButtonSet: https://developers.google.com/apps-script/reference/base/button-set
No way! MimeType: https://developers.google.com/apps-script/reference/base/mime-type
No way! Type: https://developers.google.com/apps-script/reference/maps/type
No way! MarkerSize: https://developers.google.com/apps-script/reference/maps/marker-size
No way! Format: https://developers.google.com/apps-script/reference/maps/format
N

In [27]:
os.path.basename('/hole/que/ase.js')

'ase.js'

In [28]:
import humps

In [29]:
humps.camelize("THIS_IS_A_CONSTANT")

'THIS_IS_A_CONSTANT'

In [30]:
for a in data:
    if a['type'] == 'class' and (a['name'].endswith('App') or a['name'].endswith('Service')):
        print(a['name'])

XmlService
UrlFetchApp
Service
ScriptApp
PropertiesService
MailApp
LinearOptimizationService
LockService
HtmlService
HtmlService
ContentService
ConferenceDataService
UrlFetchApp
CacheService
CardService
LockService
SpreadsheetApp
SlidesApp
SitesApp
LanguageApp
GroupsApp
GmailApp
FormApp
DriveApp
DocumentApp
DataStudioApp
ContactsApp
CalendarApp
